In [1]:
# windows only hack for graphviz path 
import os
for path in os.environ['PATH'].split(os.pathsep):
    if path.endswith("Library\\bin"):
        os.environ['PATH']+=os.pathsep+os.path.join(path, 'graphviz')

In [2]:
# 設定環境變數來控制 keras, theano
os.environ['KERAS_BACKEND']="tensorflow"
#os.environ['THEANO_FLAGS']="floatX=float32, device=cuda"

In [3]:
import keras
from keras.models import Sequential
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [4]:
import keras.backend as K
# 設定 channels_first 或 channels_last
K.set_image_data_format('channels_last')

In [5]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet',  include_top=False, input_shape=(224,224,3))

In [6]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.models import Model
_ = base_model.get_layer("block5_conv3").output
_ = GlobalAveragePooling2D()(_)
_ = Dense(512, activation='relu')(_)
_ = Dropout(0.5)(_)
# 2 個輸出
predictions = Dense(2, activation='softmax')(_)
# 這是我們的 model
model = Model(inputs=base_model.input, outputs=predictions)

# 將原來的模型設定成不可訓練，只訓練上面的兩層 dense layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
import numpy as np
def preprocess_X(X):
    return X[:,:,:,::-1].astype('float32')-[103.939,116.779,123.68]
train_X=np.zeros(shape=(20000,224,224,3), dtype="uint8")
for i in range(10000):
    img = Image.open("../Week06/train/cat.%d.jpg"%i).resize((224,224))
    train_X[2*i] =np.array(img)    
    img = Image.open("../Week06/train/dog.%d.jpg"%i).resize((224,224))    
    train_X[2*i+1] = np.array(img)
train_y=[0,1]*10000  
train_Y = np.eye(2)[train_y]

In [8]:
def generate(X, Y):    
    while 1:
        idx = np.random.choice(18000, size=32, replace=False)    
        _X = preprocess_X(X[idx])
        _Y = Y[idx]
        yield (_X, _Y)
v_data = (preprocess_X(train_X[18000:]), train_Y[18000:])
model.fit_generator(generate(train_X, train_Y), steps_per_epoch=500, epochs=10, validation_data=v_data)

Epoch 1/10
500/500 [==============================] - 169s - loss: 0.1330 - acc: 0.9692 - val_loss: 0.0755 - val_acc: 0.9775
Epoch 2/10
500/500 [==============================] - 172s - loss: 0.0561 - acc: 0.9826 - val_loss: 0.0734 - val_acc: 0.9805
Epoch 3/10
500/500 [==============================] - 172s - loss: 0.0450 - acc: 0.9846 - val_loss: 0.0604 - val_acc: 0.9840
Epoch 4/10
500/500 [==============================] - 172s - loss: 0.0340 - acc: 0.9882 - val_loss: 0.0681 - val_acc: 0.9810
Epoch 5/10
500/500 [==============================] - 172s - loss: 0.0312 - acc: 0.9894 - val_loss: 0.0617 - val_acc: 0.9835
Epoch 6/10
500/500 [==============================] - 172s - loss: 0.0324 - acc: 0.9894 - val_loss: 0.0581 - val_acc: 0.9845
Epoch 7/10
500/500 [==============================] - 173s - loss: 0.0294 - acc: 0.9914 - val_loss: 0.0530 - val_acc: 0.9860
Epoch 8/10
500/500 [==============================] - 173s - loss: 0.0315 - acc: 0.9904 - val_loss: 0.0582 - val_acc: 0.9865


### 往前微調幾層

In [9]:
for n in ["block5_conv1", "block5_conv2", "block5_conv3"]:
    model.get_layer(n).trainable = True
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])
model.fit_generator(generate(train_X, train_Y), steps_per_epoch=500, epochs=10, validation_data=v_data)

Epoch 1/10
500/500 [==============================] - 195s - loss: 0.0804 - acc: 0.9742 - val_loss: 0.0618 - val_acc: 0.9795
Epoch 2/10
 39/500 [=>............................] - ETA: 161s - loss: 0.0610 - acc: 0.9784

KeyboardInterrupt: 